* Day Count Convention	30/360
* O/N Leg Frequency	Daily
* Fixed Leg Frequency	Annual


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol


# read data
irs_data = pd.read_csv("../data/IRS_Data.csv")
irs_data.columns = map(str.lower, irs_data.columns)

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1y": 1,
    "2y": 2,
    "3y": 3,
    "4y": 4,
    "5y": 5,
    "7y": 7,
    "10y": 10,
    "15y": 15,
    "20y": 20,
    "30y": 30,
}

# OIS processing
irs_data["tenor"] = irs_data["tenor"].map(tenor_mapping)
irs_data["rate"] = irs_data["rate"].str.strip("%").astype(float) / 100.0
irs_data['tenor_diff'] = irs_data['tenor'] - irs_data['tenor'].shift() 
irs_data['tenor_diff'].fillna(0,inplace=True)
irs_data['interp_count'] = (irs_data['tenor_diff']/0.5).astype(int)
# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

In [2]:
irs_data

,tenor,product,rate,tenor_diff,interp_count
0,0.5,LIBOR,0.0250,0.0,0
1,1.0,IRS,0.0280,0.5,1
2,2.0,IRS,0.0300,1.0,2
3,3.0,IRS,0.0315,1.0,2
4,4.0,IRS,0.0325,1.0,2
5,5.0,IRS,0.0330,1.0,2
6,7.0,IRS,0.0350,2.0,4
7,10.0,IRS,0.0370,3.0,6
8,15.0,IRS,0.0400,5.0,10
9,20.0,IRS,0.0450,5.0,10


##  solve for < 1 y stuff ##

Because these 2 are anchors i believe

In [3]:
irs_data.loc[0,'disc_factor'] = 1/(1 + irs_data.loc[0,'tenor'] * irs_data.loc[0,'rate'])

In [4]:
irs_data.loc[1,'disc_factor'] = (1 - 0.5 * irs_data.loc[1,'rate'] * irs_data.loc[0,'disc_factor']) /(1 + 0.5 *irs_data.loc[1,'tenor'] * irs_data.loc[1,'rate'])

In [5]:
irs_data

,tenor,product,rate,tenor_diff,interp_count,disc_factor
0,0.5,LIBOR,0.0250,0.0,0,0.987654
1,1.0,IRS,0.0280,0.5,1,0.972557
2,2.0,IRS,0.0300,1.0,2,NaN
3,3.0,IRS,0.0315,1.0,2,NaN
4,4.0,IRS,0.0325,1.0,2,NaN
5,5.0,IRS,0.0330,1.0,2,NaN
6,7.0,IRS,0.0350,2.0,4,NaN
7,10.0,IRS,0.0370,3.0,6,NaN
8,15.0,IRS,0.0400,5.0,10,NaN
9,20.0,IRS,0.0450,5.0,10,NaN


## We may begin interpolation ##

In [6]:
previous_disc_factors = irs_data.loc[0:1,"disc_factor"].to_numpy()
print(previous_disc_factors)

[0.98765432 0.97255704]


In [7]:
# k = 6 to 10
for k in range(2,11):
    mul_fact = 1.0/irs_data.loc[k,'interp_count']
    prev_disc = previous_disc_factors[-1]
    rate = irs_data.loc[k, "rate"]
    max_sum = sum(range(1,irs_data.loc[k,'interp_count']))
    print(f"for {rate}  with prev {prev_disc} : {mul_fact} * {max_sum} = {mul_fact * max_sum}")
    denominator = 1 / (0.5 * rate * (mul_fact * max_sum + 1) + 1)

    # up till now how many disc factors do we have
    sum_prev_disc_rate = previous_disc_factors.sum()
    # add the interpolation point
    sum_prev_disc_rate += (1 - mul_fact * max_sum) * prev_disc
    print(sum_prev_disc_rate)
    numerator = 1 - (0.5 * rate* sum_prev_disc_rate)
    irs_data.loc[k, "disc_factor"] = numerator * denominator

    # find points between
    interpolated_disc_factors = []
    diff_factor = mul_fact * (irs_data.loc[k, "disc_factor"] - irs_data.loc[k-1, "disc_factor"]) 

    for x in range(1,irs_data.loc[k,'interp_count']):
        interpolated_disc_factors.append(irs_data.loc[k-1, "disc_factor"] + diff_factor * x)
    ##rint(interpolated_disc_factors)
    previous_disc_factors = np.append(previous_disc_factors, interpolated_disc_factors)
    previous_disc_factors = np.append(previous_disc_factors, irs_data.loc[k, "disc_factor"])
    #print(previous_disc_factors)


for 0.03  with prev 0.9725570409331094 : 0.5 * 1 = 0.5
2.4464898823873185
for 0.0315  with prev 0.9421052828989637 : 0.5 * 1 = 0.5
4.330700448185246
for 0.0325  with prev 0.9102859620867821 : 0.5 * 1 = 0.5
6.151272372358809
for 0.033  with prev 0.878625331494003 : 0.5 * 1 = 0.5
7.908523035346816
for 0.035  with prev 0.8485087776694584 : 0.25 * 6 = 1.5
8.757031813016274
for 0.037000000000000005  with prev 0.8112593468476312 : 0.16666666666666666 * 15 = 2.5
11.265308715202822
for 0.04  with prev 0.7434531944294415 : 0.1 * 45 = 4.5
14.510346102593347
for 0.045  with prev 0.6394532233766964 : 0.1 * 45 = 4.5
21.73687810478228
for 0.05  with prev 0.4546565006828909 : 0.05 * 190 = 9.5
25.48853438974717


In [8]:
irs_data

,tenor,product,rate,tenor_diff,interp_count,disc_factor
0,0.5,LIBOR,0.0250,0.0,0,0.987654
1,1.0,IRS,0.0280,0.5,1,0.972557
2,2.0,IRS,0.0300,1.0,2,0.942105
3,3.0,IRS,0.0315,1.0,2,0.910286
4,4.0,IRS,0.0325,1.0,2,0.878625
5,5.0,IRS,0.0330,1.0,2,0.848509
6,7.0,IRS,0.0350,2.0,4,0.811259
7,10.0,IRS,0.0370,3.0,6,0.743453
8,15.0,IRS,0.0400,5.0,10,0.639453
9,20.0,IRS,0.0450,5.0,10,0.454657


In [9]:
pd.DataFrame([np.arange(0.5,30.5,0.5),previous_disc_factors]).T

,0,1
0,0.5,0.987654
1,1.0,0.972557
2,1.5,0.957331
3,2.0,0.942105
4,2.5,0.926196
5,3.0,0.910286
6,3.5,0.894456
7,4.0,0.878625
8,4.5,0.863567
9,5.0,0.848509
